In [13]:
#!/usr/bin/env python
import bottle as bt
import returnData

##add these below to new files
import urllib2  
import json  
import sqlite3
import shutil
import os
##

##saving stdout to variable
from cStringIO import StringIO
import sys

from collections import namedtuple
from ansible.parsing.dataloader import DataLoader
from ansible.vars.manager import VariableManager
from ansible.inventory.manager import InventoryManager
from ansible.playbook.play import Play
from ansible.executor.task_queue_manager import TaskQueueManager
from ansible.plugins.callback import CallbackBase
from ansible.executor.playbook_executor import PlaybookExecutor
import ansible.constants as C

class ResultCallback(CallbackBase):
    """A sample callback plugin used for performing an action as results come in

    If you want to collect all results into a single object for processing at
    the end of the execution, look into utilizing the ``json`` callback plugin
    or writing your own custom callback plugin
    """
    def v2_runner_on_ok(self, result, **kwargs):
        """Print a json representation of the result

        This method could store the result in an instance attribute for retrieval later
        """
        host = result._host
        print(json.dumps({host.name: result._result}, indent=4))
#This file should be located in /YourApp/py/
#Therefore, the file location should be /YourApp/py/runWebAppServer.py
#Do not forget to chmod 755 this file before running on your linux server

def run_playbook(filename, pb_vars):
    print '0'
    print pb_vars
    #old_stdout = sys.stdout
    #sys.stdout = mystdout = StringIO()
    print '1'
    playbook_name = filename
    archive_path = 'playbooks/'
    playbook_folder = playbook_name+'/'
    loader = DataLoader()
    inventory_file_path = archive_path+playbook_folder+'inventory'
    
    callback = ResultCallback()
    
    #put file path of inventory file inside sources
    inventory = InventoryManager(loader=loader, sources=[inventory_file_path])
    try:
        playbook_path = archive_path + playbook_folder + playbook_name+'.yml'
    except:
        print "hi"
        
    try:
        playbook_path = archive_path + playbook_folder + playbook_name+'.yaml'
    except:
        print "hey"
    variable_manager = VariableManager(loader=loader, inventory=inventory)
    print '2'
    #print out hosts under type 'dns'
    if not os.path.exists(inventory_file_path):
        print '[INFO] The inventory does not exist'
        sys.exit()
    #print(inventory.get_groups_dict()['dns'])
    if not os.path.exists(playbook_path):
        print '[INFO] The playbook does not exist'
        sys.exit()
    #had to add 'diff' into named tuple & diff=false into Options
    Options = namedtuple('Options', ['listtags', 'listtasks', 'listhosts', 'syntax', 'connection','module_path', 'forks', 'remote_user', 'private_key_file', 'ssh_common_args', 'ssh_extra_args', 'sftp_extra_args', 'scp_extra_args', 'become', 'become_method', 'become_user', 'verbosity', 'check', 'diff'])
    options = Options(listtags=False, listtasks=False, listhosts=False, syntax=False, connection='paramiko', module_path=None, forks=100, remote_user='tbo2', private_key_file=None, ssh_common_args=None, ssh_extra_args=None, sftp_extra_args=None, scp_extra_args=None, become=None, become_method=None, become_user=None, verbosity=None, check=False, diff=False)

    #defining playbook variables in variable manager
    vars_dict = {}
    if playbook_name == 'ping':
        print('trying')
        #vars_dict['ip'] = pb_vars
        #print('printing vars dict: '+ vars_dict)
        variable_manager.extra_vars = {'ip':pb_vars} # This can accomodate various other command line arguments.
    #vars_dict.update(pb_vars)
    #print vars_dict
    elif pb_vars != None:    
        vars_dict.update(pb_vars)
        print vars_dict
        variable_manager.extra_vars = vars_dict
        print "it works"
    passwords = {}
    
    pbex = PlaybookExecutor(playbooks=[playbook_path], inventory=inventory, variable_manager=variable_manager, loader=loader, options=options, passwords=passwords)
    result = pbex.run()
    
    #sys.stdout = old_stdout
    return result

NameError: name 'QtGui' is not defined